In [13]:
#Install OpenCV

!apt-get update
!apt-get install -y libopencv-dev

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,457 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,164 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 P

In [14]:
%%writefile prog.cu

#include <stdio.h>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>  // OpenCV header

using namespace cv;  //OpenCV namespace

// CUDA kernel to convert RGB to Grayscale
__global__ void rgb_to_greyscale(unsigned char *input, unsigned char *output, int width, int height)
{
  int x = blockIdx.x * blockDim.x + threadIdx.x;
  int y = blockIdx.y * blockDim.y + threadIdx.y;
  int idx = (y*width+x)*3;                         // 3 channels (R, G, B)


  if (x < width && y < height){
    unsigned char r = input[idx];                 // Red channel
    unsigned char g = input[idx+1];               // Green channel
    unsigned char b = input[idx+2];               // Blue channel

    // Apply the formula: L = 0.21*R + 0.72*G + 0.07*B
    unsigned char L = (unsigned char)(0.21f*r + 0.72f*g + 0.07f*b);
    output[y*width+x] = L;
  }
}




int main(){
  // Load the image using OpenCV
  Mat img = imread("parrot.jpg", IMREAD_COLOR);    // Read image as BGR

  if (img.empty()) {
    printf("Error: Could not open or find the image.\n");
    return -1;
  }

  int width = img.cols;
  int height = img.rows;
  int img_size = width * height * 3;               // For RGB (or BGR) image
  int grey_img_size = width * height;              // For grayscale image

  // Allocate memory on the host for the grayscale output
  unsigned char *h_output = (unsigned char*)malloc(grey_img_size);   //Host output image (Grayscale)

  // OpenCV stores data in BGR format by default
  unsigned char *h_input = img.data;                // OpenCV Mat image data pointer (BGR)

  unsigned char *d_input, *d_output;
  cudaMalloc((void**)&d_input, img_size);
  cudaMalloc((void**)&d_output, grey_img_size);

  //create CUDA events for timing
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);

  cudaMemcpy(d_input, h_input, img_size, cudaMemcpyHostToDevice);

  dim3 threadsPerBlock(16,16);
  dim3 blocksPerGrid((width+15)/16, (height+15)/16);

  rgb_to_greyscale<<<blocksPerGrid,threadsPerBlock>>> (d_input, d_output, width, height);

  cudaMemcpy(h_output, d_output, grey_img_size, cudaMemcpyDeviceToHost);

  cudaEventSynchronize(stop);
  cudaEventRecord(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("Time for grayscale conversion: %f ms\n", milliseconds);

  Mat grey_img(height, width, CV_8UC1, h_output);
  imwrite("greyscale.jpg", grey_img);

  cudaFree(d_input);
  cudaFree(d_output);

  free(h_output);

  return 0;

}

Overwriting prog.cu


In [15]:
!nvcc -o exe prog.cu `pkg-config --cflags --libs opencv4`
!./exe

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is